In [24]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import random
import pickle
import json

In [26]:
with open('D:\\7th Semester\\PROJECT-WORK-PHASE-1\\CODEX-PROJECT-REPO\\intents.json') as file:
  data = json.load(file)
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'Hii',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye', 'Shining off'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.',
    'Had a good time! See you soon.'],
   'context_set': ''},
  {'tag': 'numbers',
   'patterns': ['one',
    'two',
    'three',
    'four',
    'five',
    'six',
    'seven',
    'eight',
    'nine',
    'zero'],
   'responses': [],
   'context_set': ''},
  {'tag': 'keywords',
   'patterns': ['False',
    'None',
    'True',
    'and',
    'as',
    'assert',
    'async',
    'await',
    'break',
    'class',
    'continue',
    'def',
    'del',
    'elif',
    'else',
    'except',
    'finally',
    'for',
    'from

In [27]:
#Initialize Stemmer
stemmer = LancasterStemmer()
tf.compat.v1.reset_default_graph()

DATA PRE-PROCESSING

In [28]:
all_words = []
intents = []
pattern_set = []
tag_set = []
for intent in data['intents']:
  for pattern in intent['patterns']:
    wrds = nltk.word_tokenize(pattern)
    all_words.extend(wrds)
    pattern_set.append(wrds)
    tag_set.append(intent['tag'])
  if intent not in intents:
    intents.append(intent['tag'])

all_words = [stemmer.stem(w.lower()) for w in all_words if w!="?"]
all_words = sorted(list(set(all_words)))
intents = sorted(intents)

In [29]:
len(all_words)
# alL_words[:10]

124

In [30]:
pattern_set

[['Hi'],
 ['Hii'],
 ['How', 'are', 'you'],
 ['Is', 'anyone', 'there', '?'],
 ['Hello'],
 ['Good', 'day'],
 ['Bye'],
 ['See', 'you', 'later'],
 ['Goodbye'],
 ['Shining', 'off'],
 ['one'],
 ['two'],
 ['three'],
 ['four'],
 ['five'],
 ['six'],
 ['seven'],
 ['eight'],
 ['nine'],
 ['zero'],
 ['False'],
 ['None'],
 ['True'],
 ['and'],
 ['as'],
 ['assert'],
 ['async'],
 ['await'],
 ['break'],
 ['class'],
 ['continue'],
 ['def'],
 ['del'],
 ['elif'],
 ['else'],
 ['except'],
 ['finally'],
 ['for'],
 ['from'],
 ['global'],
 ['if'],
 ['import'],
 ['in'],
 ['is'],
 ['lambda'],
 ['nonlocal'],
 ['not'],
 ['or'],
 ['pass'],
 ['raise'],
 ['return'],
 ['try'],
 ['while'],
 ['with'],
 ['yield'],
 ['else', 'if'],
 ['printof'],
 ['print'],
 ['charof'],
 ['intof'],
 ['floatof'],
 ['doubleof'],
 ['rangeof'],
 ['range'],
 ['isequalsto'],
 ['equals'],
 ['equals', 'to'],
 ['isequalto'],
 ['assignedto'],
 ['assign'],
 ['equalto'],
 ['lessthan'],
 ['lesserthan'],
 ['less'],
 ['greaterthan'],
 ['greater'],
 ['gre

In [31]:
len(intents)

25

In [32]:
len(data['intents'])

25

In [33]:
training_set = []
output = []

#output format - one hot encoding to decide which intent it belongs to
out_format = [0 for _ in range(len(intents))]

#creating one hot encoding for each sentence with respect to the vocabulary
for x,doc in enumerate(pattern_set):
  bag = []
  wrds = [stemmer.stem(w.lower()) for w in doc]
  for w in all_words:
    if w in wrds:
      bag.append(1)
    else:
      bag.append(0)
  output_row = out_format[:]
  output_row[intents.index(tag_set[x])]=1
  training_set.append(bag)
  output.append(output_row)

In [34]:
len(training_set[0])

124

In [35]:
len(output[0])

25

In [36]:
training_set = np.array(training_set)
output = np.array(output)

In [37]:
#Saving the training and output format as a pickle file
with open("D:\\7th Semester\\PROJECT-WORK-PHASE-1\\CODEX-PROJECT-REPO\\SAVED_MODELS\\data.pickle","wb") as f:
  pickle.dump((all_words,intents,training_set,output),f)

MODEL CONSTRUCTION

In [38]:
model = tflearn.input_data(shape=[None ,len(training_set[0])])
model = tflearn.fully_connected(model,8)
model = tflearn.fully_connected(model,8)

model = tflearn.fully_connected(model,len(output[0]),activation="softmax")
model = tflearn.regression(model)

model = tflearn.DNN(model)

In [39]:

model.fit(training_set,output,n_epoch=350,batch_size=8,show_metric=True)
model.save("model.tflearn")

Training Step: 5249  | total loss: 0.62751 | time: 0.035s
| Adam | epoch: 350 | loss: 0.62751 - acc: 0.8215 -- iter: 112/120
Training Step: 5250  | total loss: 0.64970 | time: 0.039s
| Adam | epoch: 350 | loss: 0.64970 - acc: 0.8144 -- iter: 120/120
--
INFO:tensorflow:d:\7th Semester\PROJECT-WORK-PHASE-1\CODEX-PROJECT-REPO\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [40]:
tf.compat.v1.reset_default_graph()

MODEL PREDICTION

In [41]:
#To pre-process the input text
def bag_of_words(inp,words):
  bag = [0 for _ in range(len(words))]
  user_words = nltk.word_tokenize(inp)
  user_words = [stemmer.stem(word.lower()) for word in user_words]

  for element in user_words:
    for i,w in enumerate(words):
      if w == element:
        bag[i]=1
  return bag

In [42]:
number_map={
    "one":'1',
    "two":'2',
    "three":'3',
    "four":'4',
    "five":'5',
    "six":'6',
    "seven":'7',
    "eight":'8',
    "nine":'9',
    "zero":'0'
}

In [43]:
def predict(input):
  results = model.predict([bag_of_words(input,all_words)])
  result_index = np.argmax(results)
  tag = intents[result_index]
  print(tag,end=" ")
  print(results[0][result_index])
  if results[0][result_index]>0.8:
    if tag == "keywords":
      return input
    elif tag=="else_if":
      return "elif"
    elif tag == "inbuilt_functions":
      input = input.replace('of','')
      input+="("
      return input
    elif tag=="numbers":
      if input in number_map:
        return number_map[input]
      else:
        return input
    elif tag == "greeting" or tag=="goodbye":
      for t in data['intents']:
        if t['tag']==tag:
          responses = t['responses']
          return(random.choice(responses))
    else:
      for t in data['intents']:
        if t['tag']==tag:
          responses = t['responses']
          return(responses[0])
  else:
    print(results[0][result_index])
    return input

In [44]:
def preprocess(string):
    string = string.replace('equals equals','equalsequals').replace('slash slash','slashslash').replace('is equals to','isequalsto').replace('is equal to','isequalto').replace('less than or equals to','lessthanorequalsto').replace('less than equals to','lessthanequalsto').replace('greater than or equals to','greaterthanorequalsto').replace('greater than equals to','greaterthanequalsto').replace('equals to','equalsto').replace('equalto','equalto').replace('less than or','lessthanor').replace('greater than or','greaterthanor').replace('less than','lessthan').replace('greater than','greaterthan').replace('else if','elseif').replace('range of','rangeof').replace('print of','printof')
    return string

In [45]:
print(preprocess("for i in range of five"))
print(preprocess("if i equals to three"))
print(preprocess("else if i less than or equals to nine"))

for i in rangeof five
if i equalsto three
elseif i lessthanorequalsto nine


PARSE ALGORITHM

In [46]:
# bag_of_words("for",all_words).count(1)
def parse_statement(string):
  string = preprocess(string)
  print("Pre-processed String: ",string)
  result = []
  for word in string.split(' '):
    print(word,end=" ")
    result.append(predict(word))
    # print(predict(word))
    print(result)
  return' '.join(result)
# string = input('Enter Your String : ')
# parse_statement(string)